# Pix2Pix Caricature Generation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/USERNAME/REPO/blob/main/aicaricaturist/pix2pix_caricature/caricature_training.ipynb)

This notebook implements a Pix2Pix model for face-to-caricature translation using PyTorch.

In [ ]:
# Install required packages
!pip install torch torchvision

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.utils import save_image
import os
import glob
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import drive
from datetime import datetime

## Mount Google Drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Set data directory
DATA_DIR = '/content/drive/MyDrive/caricature Project Diffusion/paired_caricature'
CHECKPOINT_DIR = '/content/drive/MyDrive/caricature_checkpoints'

# Create checkpoint directory if it doesn't exist
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data Loading and Preprocessing

In [ ]:
class CaricatureDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.face_paths = sorted(glob.glob(os.path.join(root_dir, '*_f.png')))
        self.caricature_paths = sorted(glob.glob(os.path.join(root_dir, '*_c.png')))
        
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        
        # Augmentation parameters
        self.pad_size = 51  # 10% of 512
    
    def __len__(self):
        return len(self.face_paths)
    
    def __getitem__(self, idx):
        face_path = self.face_paths[idx]
        caricature_path = self.caricature_paths[idx]
        
        face = Image.open(face_path).convert('RGB')
        caricature = Image.open(caricature_path).convert('RGB')
        
        # Random horizontal flip
        if random.random() > 0.5:
            face = transforms.functional.hflip(face)
            caricature = transforms.functional.hflip(caricature)
        
        # Random translation
        pad = transforms.Pad(self.pad_size, padding_mode='reflect')
        face = pad(face)
        caricature = pad(caricature)
        
        # Random crop
        i = random.randint(0, 2 * self.pad_size)
        j = random.randint(0, 2 * self.pad_size)
        face = transforms.functional.crop(face, i, j, 512, 512)
        caricature = transforms.functional.crop(caricature, i, j, 512, 512)
        
        return self.transform(face), self.transform(caricature)

# Create dataset and dataloader
dataset = CaricatureDataset(DATA_DIR)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=2)

## Model Implementation

In [ ]:
def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

class UNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, normalize=True, dropout=0.0):
        super().__init__()
        layers = [nn.Conv2d(in_channels, out_channels, 4, stride=2, padding=1, bias=False)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.LeakyReLU(0.2))
        if dropout > 0:
            layers.append(nn.Dropout(dropout))
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)

class UNetUpBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.0):
        super().__init__()
        layers = [
            nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        ]
        if dropout > 0:
            layers.append(nn.Dropout(dropout))
        self.block = nn.Sequential(*layers)

    def forward(self, x, skip):
        x = self.block(x)
        x = torch.cat([x, skip], 1)
        return x

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Encoder
        self.down1 = UNetBlock(3, 64, normalize=False)
        self.down2 = UNetBlock(64, 128)
        self.down3 = UNetBlock(128, 256)
        self.down4 = UNetBlock(256, 512)
        self.down5 = UNetBlock(512, 512)
        self.down6 = UNetBlock(512, 512)
        self.down7 = UNetBlock(512, 512)
        self.down8 = UNetBlock(512, 512)
        
        # Decoder
        self.up1 = UNetUpBlock(512, 512, dropout=0.5)
        self.up2 = UNetUpBlock(1024, 512, dropout=0.5)
        self.up3 = UNetUpBlock(1024, 512, dropout=0.5)
        self.up4 = UNetUpBlock(1024, 512)
        self.up5 = UNetUpBlock(1024, 256)
        self.up6 = UNetUpBlock(512, 128)
        self.up7 = UNetUpBlock(256, 64)
        
        self.final = nn.Sequential(
            nn.ConvTranspose2d(128, 3, 4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        d8 = self.down8(d7)
        
        u1 = self.up1(d8, d7)
        u2 = self.up2(u1, d6)
        u3 = self.up3(u2, d5)
        u4 = self.up4(u3, d4)
        u5 = self.up5(u4, d3)
        u6 = self.up6(u5, d2)
        u7 = self.up7(u6, d1)
        
        return self.final(u7)

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        def discriminator_block(in_channels, out_channels, normalize=True):
            layers = [nn.Conv2d(in_channels, out_channels, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.LeakyReLU(0.2))
            return layers
        
        self.model = nn.Sequential(
            *discriminator_block(6, 64, normalize=False),  # Input: face + caricature
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        )

    def forward(self, face, caricature):
        img_input = torch.cat([face, caricature], 1)
        return self.model(img_input)

# Initialize models
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Initialize weights
generator.apply(init_weights)
discriminator.apply(init_weights)

# Loss functions
criterion_GAN = nn.BCEWithLogitsLoss()
criterion_L1 = nn.L1Loss()

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

## Training

In [ ]:
def train_model(num_epochs=200):
    for epoch in range(num_epochs):
        for i, (faces, caricatures) in enumerate(dataloader):
            faces = faces.to(device)
            caricatures = caricatures.to(device)
            
            # Ground truths
            valid = torch.ones((faces.size(0), 1, 30, 30), requires_grad=False).to(device)
            fake = torch.zeros((faces.size(0), 1, 30, 30), requires_grad=False).to(device)
            
            # Train Generator
            optimizer_G.zero_grad()
            gen_caricatures = generator(faces)
            
            # GAN loss
            pred_fake = discriminator(faces, gen_caricatures)
            loss_GAN = criterion_GAN(pred_fake, valid)
            
            # L1 loss
            loss_L1 = criterion_L1(gen_caricatures, caricatures)
            
            # Total loss
            loss_G = loss_GAN + 100 * loss_L1
            loss_G.backward()
            optimizer_G.step()
            
            # Train Discriminator
            optimizer_D.zero_grad()
            
            # Real loss
            pred_real = discriminator(faces, caricatures)
            loss_real = criterion_GAN(pred_real, valid)
            
            # Fake loss
            pred_fake = discriminator(faces, gen_caricatures.detach())
            loss_fake = criterion_GAN(pred_fake, fake)
            
            # Total discriminator loss
            loss_D = (loss_real + loss_fake) / 2
            loss_D.backward()
            optimizer_D.step()
            
            if i % 100 == 0:
                print(f'[Epoch {epoch}/{num_epochs}] [Batch {i}/{len(dataloader)}] '
                      f'[D loss: {loss_D.item():.4f}] [G loss: {loss_G.item():.4f}]')
        
        if (epoch + 1) % 20 == 0:
            # Save model checkpoints
            torch.save({
                'epoch': epoch,
                'generator_state_dict': generator.state_dict(),
                'discriminator_state_dict': discriminator.state_dict(),
                'optimizer_G_state_dict': optimizer_G.state_dict(),
                'optimizer_D_state_dict': optimizer_D.state_dict(),
            }, os.path.join(CHECKPOINT_DIR, f'checkpoint_epoch_{epoch}.pt'))

# Train the model
train_model()

## Inference

In [ ]:
def generate_caricature(face_path, checkpoint_path):
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path)
    generator.load_state_dict(checkpoint['generator_state_dict'])
    generator.eval()
    
    # Load and preprocess image
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    face = Image.open(face_path).convert('RGB')
    face = transform(face).unsqueeze(0).to(device)
    
    with torch.no_grad():
        generated = generator(face)
        generated = (generated * 0.5 + 0.5).clamp(0, 1)
        generated = generated.squeeze().permute(1, 2, 0).cpu().numpy()
    
    return generated

# Example usage:
# checkpoint_path = os.path.join(CHECKPOINT_DIR, 'checkpoint_epoch_199.pt')
# generated = generate_caricature('path_to_face.png', checkpoint_path)
# plt.imshow(generated)
# plt.axis('off')
# plt.show()